# VAE

In [1]:
import os
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, LeakyReLU
from tensorflow.keras.layers import Reshape, Conv2DTranspose, UpSampling2D
from tensorflow.keras import Model
from tensorflow import keras
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import tensorflow as tf
import moviepy.editor as mp
import numpy as np
import os

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

2023-03-14 16:55:21.519330: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-14 16:55:22.043555: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-14 16:55:22.151417: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-14 16:55:22.151441: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not ha

In [2]:
from tensorflow.keras.utils import image_dataset_from_directory

In [10]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
    # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
              tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [11]:
ds = image_dataset_from_directory('zelle', label_mode=None, image_size=(448, 448), batch_size=16)

Found 253 files belonging to 1 classes.


2023-03-14 15:38:28.490211: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
foldername = 'zelle'

def load_images_from_folder(foldername, output_folder):
    latent_space = []
    for filename in os.listdir(foldername):
        with Image.open(os.path.join(foldername,filename)).resize((448,448)) as img:
            if img is not None:
                img = np.asarray(img)
                encoded = vae.encoder(img.reshape(-1,448,448,3)/255)
                latent_space.append(encoded[2])
    return latent_space
           
latent_space = load_images_from_folder(foldername, "encoded_images")

# images/255



In [14]:
from keras.layers import Lambda

In [5]:
from keras.models import Model
from keras.losses import binary_crossentropy
from keras.layers import Layer

In [6]:
import tensorflow as tf
class Sampling(tf.keras.layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.random.normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [119]:
latent_dim = 200


input_image = Input(shape=(448, 448, 3))
    
x = Conv2D(32, (3, 3), padding='same', activation="relu")(input_image)
#x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(64, (3, 3), padding='same', activation="relu")(x)
#x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(128, (3, 3), padding='same', activation="relu")(x)
#x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)

x = Conv2D(256, (3, 3), padding='same', activation="relu")(x)
#x = BatchNormalization()(x)
x = MaxPooling2D((2, 2))(x)

x = Flatten()(x)
encoder_output = Dense(latent_dim, activation="relu")(x)

z_mean = tf.keras.layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = tf.keras.layers.Dense(latent_dim, name="z_log_var")(x)

z = Sampling()([z_mean, z_log_var])
encoder = tf.keras.Model(input_image, [z_mean, z_log_var, z], name="encoder")

#encoder.summary()

In [120]:
latent_inputs = tf.keras.Input(shape=(latent_dim,))  
x = Dense(7*7*64, activation='tanh')(latent_inputs)
x = Reshape((7, 7, 64))(x)
x = Conv2DTranspose(128, (3, 3), strides=2, padding='same', activation="relu")(x)
x = Conv2DTranspose(128, (3, 3), strides=1, padding='same', activation="relu")(x)
x = Conv2DTranspose(128, (3, 3), strides=1, padding='same', activation="relu")(x)
x = Conv2DTranspose(128, (3, 3), strides=1, padding='same', activation="relu")(x)
x = Conv2DTranspose(64, (3, 3), strides=2, padding='same', activation="relu")(x)
x = Conv2DTranspose(64, (3, 3), strides=1, padding='same', activation="relu")(x)
x = Conv2DTranspose(64, (3, 3), strides=1, padding='same', activation="relu")(x)
x = Conv2DTranspose(32, (3, 3), strides=2, padding='same', activation="relu")(x)
x = Conv2DTranspose(32, (3, 3), strides=1, padding='same', activation="relu")(x)
x = Conv2DTranspose(32, (3, 3), strides=1, padding='same', activation="relu")(x)
x = Conv2DTranspose(16, (3, 3), strides=2, padding='same', activation="relu")(x)
x = Conv2DTranspose(16, (3, 3), strides=1, padding='same', activation="relu")(x)
x = Conv2DTranspose(16, (3, 3), strides=1, padding='same', activation="relu")(x)
x = Conv2DTranspose(8, (3, 3), strides=2, padding='same', activation="relu")(x)
x = Conv2DTranspose(8, (3, 3), strides=1, padding='same', activation="relu")(x)
x = Conv2DTranspose(8, (3, 3), strides=1, padding='same', activation="relu")(x)


    
decoder_output = Conv2DTranspose(3, (3, 3), strides=2, padding='same', activation='sigmoid')(x)
    
decoder = Model(inputs=latent_inputs, outputs=decoder_output, name="decoder")
#decoder.summary()
    

In [9]:
class VAE(tf.keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = tf.keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = tf.keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = tf.keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    tf.keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [28]:
class CallbackSaveModel(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if epoch % 10 == 0:
            self.model.save_weights(f'/home/jupyter/model_{epoch}')

save_model = CallbackSaveModel()

In [10]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=tf.keras.optimizers.Adam())

In [27]:
vae.load_weights('model_vae_2500') #load_weights

# Image loading

In [42]:
loaded_images = load_images_from_folder(foldername)

In [43]:
# norm_images = []
# for i in loaded_images:
#     norm_images.append(i/255)

In [45]:
# #all images in latent space (i.e encoded all images)  
# encoded_images = []
# for image in norm_images:
    
#     encoded_images.append(vae.encoder(image.reshape(-1,448,448,3)))

In [61]:
# decoded_images = []
# for image in encoded_images:
#     decoded_images.append(vae.decoder(image[2]))

# VAE reconstructed image

In [112]:
hop_length = 512
fps = 24


import librosa
import math

y, sr = librosa.load('Drums.wav', sr=22050)

audio_duration = librosa.get_duration(y=y, sr=sr)

total_frames_float = audio_duration * fps
total_frames = math.ceil(total_frames_float) 

sample_rate = round(total_frames_float / audio_duration * hop_length)

y, sr = librosa.load('Drums.wav', sr=sample_rate)

audio_duration = librosa.get_duration(y=y, sr=sr)
onset_strengths = librosa.onset.onset_strength(y=y, sr=sr)
onset_times = librosa.times_like(onset_strengths, sr=sr)
# calculate desired hop length based on 24 fps

In [116]:
audio_duration

8.347900390625

In [118]:
# onset_times

In [114]:
1 / fps

0.041666666666666664

In [90]:
# calculate optimal n_fft length based on hop length
n_fft = 2 ** (hop_length - 1).bit_length()

# ENVELOPES WITH HOP LENGTH DETERMINED BY FPS
onset_env = librosa.onset.onset_strength(y=y, sr=sr, hop_length = hop_length, n_fft=n_fft)

onset_env_norm = librosa.util.normalize(onset_env)

onset_frames = librosa.onset.onset_detect(y=y, sr=sr, units='time')
len(onset_frames)

N_STEPS = len(onset_env)

frame_duration = 1/fps
audio_movement = []

times = librosa.times_like(onset_env, sr=24)

IMAGE_TIME_GAP = 5 # seconds, play with this
IMAGE_FRAME_GAP = 5 * fps
N_IMAGES = 10
N_STEPS_IMAGES = N_STEPS / N_IMAGES 

# random_start_point = np.random.randint(0, len(encoded_images), size=(1))[0]
# random_start_point

In [91]:
N_STEPS_IMAGES

20.1

# Audio and video

In [54]:
def get_audio_params(audio_path, frame_rate=fps, hop_window=hop_length):
    # Load song with 22050 sample rate
    y, sr = librosa.load(audio_path, sr=22050)
    audio_duration = librosa.get_duration(y, sr=sr)
    
    total_frames_float = audio_duration * frame_rate
    total_frames = math.ceil(total_frames_float)
    
    sample_rate = round(total_frames_float / audio_duration * hop_window)
    
    return total_frames, sample_rate, audio_duration

def get_onset_info(audio_path, sample_rate, frame_rate=fps, hop_window=hop_length):
    # Load
    y, sr = librosa.load(audio_path, sr=sample_rate)
    
    # Onset strengths and normalize
    onset_strengths = librosa.onset.onset_strength(y=y, sr=sample_rate, aggregate=np.median)
    onset_strengths = librosa.util.normalize(onset_strengths)
    
    # Onset timestamps and frames
    onset_times = librosa.times_like(onset_strengths, sr=sample_rate)
    onset_frames = onset_times * frame_rate
    
    onset_info = np.concatenate([
        onset_frames.reshape((-1, 1)),
        onset_times.reshape((-1, 1)),
        onset_strengths.reshape((-1, 1))
    ], axis=1)
    
    # Beat times
    beat_times = librosa.beat.beat_track(y=y, sr=sample_rate, units='time')[1]
    
    return y, onset_info, beat_times

In [56]:
total_frames, sample_rate, audio_duration = get_audio_params('Drums.wav', frame_rate=fps, hop_window=hop_length)
total_frames, sample_rate, audio_duration

TypeError: get_duration() takes 0 positional arguments but 1 positional argument (and 1 keyword-only argument) were given

# Latent


In [53]:
len(librosa.times_like(onset_env, sr=sr))

201

In [121]:
def frames(latent_spaces):
    interpolation_images = []
    for j in range(N_IMAGES):
        start_lat_vec, end_lat_vec = latent_space[j], latent_space[j+1]
        step_vec = (end_lat_vec - start_lat_vec ) / N_STEPS_IMAGES 
        for i in range(0, N_STEPS_IMAGES):
            interpolation_images.append(start_lat_vec + (i * (step_vec *(onset_env_norm[i] + 1 * 2000))))
            #print(onset_env[i])
#         inter_images_decoded = vae.decoder(interpolation_array)
#         inter_images = (inter_images_decoded.numpy() * 255).astype('uint8')
    return np.array(interpolation_images)

PASS_THIS_TO_THE_VIDEO_GENERATING_FUNCTION = frames(latent_space)

TypeError: 'float' object cannot be interpreted as an integer

In [ ]:
    total frames / fixed frames

In [81]:
PASS_THIS_TO_THE_VIDEO_GENERATING_FUNCTION.shape

(195, 1, 200)

In [82]:
frame_duration = 1/24

In [83]:
create_video_with_audio(PASS_THIS_TO_THE_VIDEO_GENERATING_FUNCTION, audio_file_path="Drums.wav", output_video_path="./", frame_duration=frame_duration)

Moviepy - Building video output_video.mp4.
MoviePy - Writing audio in output_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output_video.mp4



Moviepy - Done !
Moviepy - video ready output_video.mp4


In [ ]:
for i in range(inter_images.shape[0]):
    im = Image.fromarray(inter_images[i])
    im.save(f'./images/image_{i}.jpg')

# Load the audio file
audio_clip = mp.AudioFileClip('Piano.wav')

# Define the frame folder where the images are stored
frame_folder = './images'

# Get the list of image paths
im_paths = [os.path.join(frame_folder, f'image_{i}.jpg') for i in range(N_STEPS)]

# Load the images as frames
frames = [mp.ImageClip(image_path).set_duration(frame_duration) # equal to 24fps
          for image_path in im_paths]

# Combine the frames into a video
video_clip = mp.concatenate_videoclips(frames, method='chain')  # experiment with different method values

# Overlay the audio on the video
final_clip = video_clip.set_audio(audio_clip)

# Write the final video to file
final_clip.write_videofile("Aoutput_video.mp4", fps=24) #

# Functions

In [ ]:
from typing import Tuple
import numpy as np
import librosa
import os
from PIL import Image
import moviepy.editor as mp

def generate_latents(path_to_audio: str, vae, fps: int = 24, frame_folder: str = './images') -> None:
    """
    Generates a video with frames interpolated between two latent vectors of a VAE, based on the onset
    strength of an audio file.

    Args:
        path_to_audio (str): The path to the audio file
        vae: The VAE model used for image generation
        fps (int): The frames per second for the output video (default 24)
        frame_folder (str): The folder path to save the frames (default './images')
    
    Returns:
        None
    """
    # load audio and sample rate
    y, sr = librosa.load(path_to_audio)

    # calculate desired hop length based on 24 fps
    hop_length = int(sr / fps)

    # calculate optimal n_fft length based on hop length
    n_fft = 2 ** (hop_length - 1).bit_length()

    # calculate onset envelopes and normalize 
    onset_env = librosa.onset.onset_strength(y=y, sr=sr, hop_length=hop_length, n_fft=n_fft)
    onset_env_norm = librosa.util.normalize(onset_env)

    # get number of steps (frames)
    n_steps = len(onset_env)

    # generate encoded images for interpolation
    #def create_step_vecs(N_IMAGES, IMAGE_FRAME_GAP, encoded_images):
    step_vecs = []
    for i in range(2):
        step_vecs.append((encoded_images[2][i] - encoded_images[2][i+1]) / n_steps)

    # interpolate between start and end latent vectors based on onset strength
    interpolation_images = []
    for i in range(n_steps):
        interpolation_images.append(start_lat_vec + (i * (step_vec * (onset_env_norm[i] + 1))))
    interpolation_array = np.array(interpolation_images)

    # decode interpolated images
    inter_images_decoded = vae.decoder(interpolation_array)
    inter_images = (inter_images_decoded.numpy() * 255).astype('uint8')
    

    # save images to frame folder
    for i in range(inter_images.shape[0]):
        im = Image.fromarray(inter_images[i])
        im.save(os.path.join(frame_folder, f'image_{i}.jpg'))

In [ ]:
path_to_audio = "Drums.wav"

frame_folder = './images'

generate_latents(path_to_audio=path_to_audio, vae=vae, fps=fps, frame_folder=frame_folder)

In [78]:
1/24

0.041666666666666664

In [57]:
# import os
# from typing import List
# import numpy as np
# import moviepy.editor as mp
# from PIL import Image


def create_video_with_audio(inter_images: np.ndarray, audio_file_path: str, output_video_path: str,
                            frame_folder: str = './images', fps: int = 24, frame_duration: float = 1.0) -> None:
    """
    Creates a video clip by concatenating a series of images with an audio clip.
    
    Parameters:
    -----------
    inter_images: numpy.ndarray
        Array of images to be used as frames in the video clip.
    audio_file_path: str
        Path to the audio file to be overlaid on the video clip.
    output_video_path: str
        Path to save the output video file.
    frame_folder: str, optional (default='./images')
        Folder where the image frames are stored.
    fps: int, optional (default=24)
        Frames per second to use when creating the video clip.
    frame_duration: float, optional (default=1.0)
        Duration of each frame in seconds.
    
    Returns:
    --------
    None
    """
    
    # Validate inputs
    if not isinstance(inter_images, np.ndarray):
        raise TypeError("Input argument 'inter_images' must be a numpy.ndarray.")
    
    if not isinstance(audio_file_path, str):
        raise TypeError("Input argument 'audio_file_path' must be a string.")
    
    if not isinstance(output_video_path, str):
        raise TypeError("Input argument 'output_video_path' must be a string.")
    
    if not isinstance(frame_folder, str):
        raise TypeError("Input argument 'frame_folder' must be a string.")
    
    if not isinstance(fps, int):
        raise TypeError("Input argument 'fps' must be an integer.")
    
    if not isinstance(frame_duration, (int, float)):
        raise TypeError("Input argument 'frame_duration' must be a numeric value.")
    
    if not os.path.exists(frame_folder):
        raise ValueError(f"Frame folder '{frame_folder}' does not exist.")
    
    if not os.path.exists(audio_file_path):
        raise ValueError(f"Audio file '{audio_file_path}' does not exist.")
    
    # save images to frame folder
    for i in range(inter_images.shape[0]):
        decoded = (vae.decoder(inter_images[i]).numpy().reshape(448,448,3) * 255).astype(np.uint8)
        im = Image.fromarray(decoded)
        im.save(os.path.join(frame_folder, f'image_{i}.jpg'))
    
    # Create frames from the input images
    im_paths = [os.path.join(frame_folder, f'image_{i}.jpg') for i in range(inter_images.shape[0])]
    frames = [mp.ImageClip(image_path).set_duration(frame_duration) for image_path in im_paths]
    
    # Load the audio file
    audio_clip = mp.AudioFileClip(audio_file_path)
    
    # Combine the frames into a video clip
    video_clip = mp.concatenate_videoclips(frames, method='chain')
    
    # Overlay the audio on the video clip
    final_clip = video_clip.set_audio(audio_clip)
    
    # Write the final video clip to file
    final_clip.write_videofile("output_video.mp4", fps=fps, codec='libx264')

In [ ]:
def create_onset_features(onset_info, beat_times, fixed_decay_frames=20, exp_decay_rate=0.2, decay_magnification=False):
    
    # Create column of zeroes as default value for both Linear and Exp Decay
    onset_info = np.concatenate([onset_info, np.zeros((onset_info.shape[0], 1)), np.zeros((onset_info.shape[0], 1))], axis=1)
    
    # for each row
    for i in range(onset_info.shape[0]):
        # Skip the first value, makes life easy
        if i == 0:
            onset_info[i, 3] = 0
        
        
        ## LINEAR
        # If the timestamp is in beat_times, it's a peak
        if onset_info[i, 1] in beat_times:
            onset_info[i, 3] = onset_info[i, 2] # Linear Column
            
            # Decay Params
            decay_factor = 1
            if decay_magnification:
                decay_factor *= (onset_info[i, 2] + 1)
            
            decay_frames = fixed_decay_frames * decay_factor
            lin_decay_val = onset_info[i, 2] / decay_frames
        
        # Check if the previous value is zero or less than the decay_val -> 0
        if onset_info[i - 1, 3] == 0. or abs(onset_info[i -1, 3]) < lin_decay_val:
            pass
        
        # If previous value > 0, needs decay
        elif onset_info[i - 1, 3] > 0: 
            onset_info[i, 3] = onset_info[i - 1, 3] - lin_decay_val
        
        
        # EXPONENTIAL
        # If the timestamp is in beat_times, it's a peak
        if onset_info[i, 1] in beat_times:
            onset_info[i, 4] = onset_info[i, 2] # Exp Column
        
        # Set current to zero if previous is zero or small number
        elif onset_info[i - 1, 4] == 0 or onset_info[i - 1, 4] < 0.005: 
            pass
        
        # If previous value > 0, needs decay
        elif onset_info[i - 1, 4] != 0: 
                onset_info[i, 4] = onset_info[i - 1, 4] * (1 - exp_decay_rate)
    
    return onset_info

In [ ]:

def get_onset_info(audio_path, sample_rate, frame_rate=fps, hop_window=hop_length):
    # Load
    y, sr = librosa.load(audio_path, sr=sample_rate)
    
    # Onset strengths and normalize
    onset_strengths = librosa.onset.onset_strength(y=y, sr=sample_rate, aggregate=np.median)
    onset_strengths = librosa.util.normalize(onset_strengths)
    
    # Onset timestamps and frames
    onset_times = librosa.times_like(onset_strengths, sr=sample_rate)
    onset_frames = onset_times * frame_rate
    
    onset_info = np.concatenate([
        onset_frames.reshape((-1, 1)),
        onset_times.reshape((-1, 1)),
        onset_strengths.reshape((-1, 1))
    ], axis=1)
    
    # Beat times
    beat_times = librosa.beat.beat_track(y=y, sr=sample_rate, units='time')[1]
    
    return y, onset_info, beat_times

In [ ]:
y, onset_info, beat_times = get_onset_info('Drums.wav', sample_rate=sr, frame_rate=fps, hop_window=hop_length)

onset_info = create_onset_features(onset_info, beat_times, fixed_decay_frames=20, exp_decay_rate=0.10, decay_magnification=False)

In [ ]:


generate_video(path_to_audio='Drums.wav', vae=vae, fps=24, frame_folder='./images')


# All audio/video functionality in one cell

In [ ]:
# Both functions in one 


# from typing import Tuple
# import numpy as np
# import librosa
# import os
# from PIL import Image
# import moviepy.editor as mp

def generate_video(path_to_audio: str, vae, fps: int = 24, frame_folder: str = './images') -> None:
    """
    Generates a video with frames interpolated between two latent vectors of a VAE, based on the onset
    strength of an audio file.

    Args:
        path_to_audio (str): The path to the audio file
        vae: The VAE model used for image generation
        fps (int): The frames per second for the output video (default 24)
        frame_folder (str): The folder path to save the frames (default './images')
    
    Returns:
        None
    """
    # load audio and sample rate
    y, sr = librosa.load(path_to_audio)

    # calculate desired hop length based on 24 fps
    hop_length = int(sr / fps)

    # calculate optimal n_fft length based on hop length
    n_fft = 2 ** (hop_length - 1).bit_length()

    # calculate onset envelopes and normalize 
    onset_env = librosa.onset.onset_strength(y=y, sr=sr, hop_length=hop_length, n_fft=n_fft)
    onset_env_norm = librosa.util.normalize(onset_env)

    # get number of steps (frames)
    n_steps = len(onset_env)

    # generate encoded images for interpolation
    encoded_images = vae.encoder(imgs[10:12].reshape(-1, 448, 448, 3))
    start_lat_vec, end_lat_vec = encoded_images[0][0], encoded_images[0][1]
    step_vec = (end_lat_vec - start_lat_vec) / n_steps

    # interpolate between start and end latent vectors based on onset strength
    interpolation_images = []
    for i in range(n_steps):
        interpolation_images.append(start_lat_vec + (i * (step_vec * (onset_env_norm[i] + 1))))
    interpolation_array = np.array(interpolation_images)

    # decode interpolated images
    inter_images_decoded = vae.decoder(interpolation_array)
    inter_images = (inter_images_decoded.numpy() * 255).astype('uint8')

    # save images to frame folder
    for i in range(inter_images.shape[0]):
        im = Image.fromarray(inter_images[i])
        im.save(os.path.join(frame_folder, f'image_{i}.jpg'))

    # load the audio clip
    audio_clip = mp.AudioFileClip(path_to_audio)

    # get the list of image paths
    im_paths = [os.path.join(frame_folder, f'image_{i}.jpg') for i in range(n_steps)]

    # load the images as frames
    frames = [mp.ImageClip(image_path).set_duration(1/fps) for image_path in im_paths]

    # combine the frames into a video
    video_clip = mp.concatenate_videoclips(frames, method='chain')

    # overlay the audio on the video
    final_clip = video_clip.set_audio(audio_clip)

    # write the final video to file
    final_clip.write_videofile("output_video.mp4", fps=fps, codec='libx264')